# PAL2 Single Pulsar Continuous Gravitational Wave Analysis Example

Nicholas West

06/15/2022

In [2]:
# Imports
# All of these requirements can be satisfied by following the installation instructions
# in the appendix of my report. Alternatively, follow the installation instructions 
# in the README for PAL2 on its github

from __future__ import division
import numpy as np
import PAL2
from PAL2 import PALmodels
from PAL2 import PALutils
from PAL2 import PALdatafile
from PAL2 import PALInferencePTMCMC as ptmcmc
from PAL2 import bayesutils as bu
import glob
import matplotlib.pyplot as plt
from corner import corner
from scipy.optimize import brentq

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [3]:
# Loading the data
datadir = "./data"
parFiles = glob.glob(datadir + '/*.par')
timFiles = glob.glob(datadir + '/*.tim')
print(datadir)
print(parFiles)
print(timFiles)
# sort 
parFiles.sort()
timFiles.sort()

./J0613_subband_MJD56733
['./J0613_subband_MJD56733/par2/J0613-0200_NANOGrav_9yv1-Copy1.gls.par']
['./J0613_subband_MJD56733/tim2/guppi_56733_J0613-0200_0002_all_allmed_mow_122f16-Copy1.tim']


In [4]:
# Create an hdf5 data file from the par and tim files--this is what PAL2 reads
h5filename = './data/h5file.hdf5'
df = PALdatafile.DataFile(h5filename)
# add pulsars to file
for p, t in zip(parFiles, timFiles):
    print '\n', t, p, '\n'
    df.addTempoPulsar(p, t, iterations=0)


./J0613_subband_MJD56733/tim2/guppi_56733_J0613-0200_0002_all_allmed_mow_122f16-Copy1.tim ./J0613_subband_MJD56733/par2/J0613-0200_NANOGrav_9yv1-Copy1.gls.par 



In [17]:
# This generates the noise model for the pulsar and saves the relevant files
# in the --outDir directory
# Params: 
# --h5file path_to_hdf5_data 
# --outDir path_to_noise_directory
# --nf number_of_frequencies_to_be_searched
# --incRed (flag) include red noise parameters
# --niter number_of_iterations_in_noise_chain
# --incEquad (flag) include EQUAD noise parameter
# --incJitterEquad (flag) include jitter EQUAD parameter
# --mark6 (flag) using mark6loglikelihood
# --pulsar name_of_pulsar
!PAL2_run.py --h5File ./data/h5file.hdf5 --outDir ./noise --nf 50 --incRed --niter 100000 --incEqua --incJitterEquad --mark6 --pulsar J0613-0200

/opt/conda/envs/python2/lib/python2.7/site-packages/PAL2-2015.4-py2.7.egg/PAL2/PALdatafile.py:142: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  data = datGroup[field].value
Requested to re-create the Auxiliaries
Creating Auxiliaries for J0613-0200
5 Free parameters
Search Parameters: ['efac-Rcvr1_2_GUPPI', 'jitter_q-Rcvr1_2_GUPPI', 'equad-Rcvr1_2_GUPPI', 'RN-Amplitude', 'RN-spectral-index']
Running model with no GWB correlations
18779.88692543822 -9.73572600568059
[[0, 1, 2, 3, 4], [0], [2], [1], [3, 4]]
Engage!
Finished 10.00 percent in 137.032947 s Acceptance rate = 0.56297Adding DE jump with weight 50
Finished 33.00 percent in 474.037526 s Acceptance rate = 0.576727
Run Complete with 1026 effective samples


In [18]:
# I have created this class to keep track of the parameters used in this analysis.
# One can change the values here to alter them in most other areas of the analysis below
class my_args_class:
    def __init__(self):
        self.h5file = './data/h5file.hdf5' # subband_hdf5 file
        self.noisedir = "./noise"# noisedir
        self.outdir = "./outputs" # output directory (hfcw_analysis)
        self.psr = "J0613-0200" # pulsar name
        self.frqres = 50 # how many frequencies
        self.nreal = 50000 # how many simulations
        self.run_name = "test" # run_name
        self.fmin = 8e-5 # min frequency to check
        self.fmax = 1e-3 # max frequency to check
        
fmin = 8e-5
fmax = 1e-3

In [19]:
args = my_args_class()

In [20]:
def upperLimitFunc(h, psr, R, fstat_ref, freq, nreal, theta=None, 
                   phi=None, inc=None, detect=False, dist=None):
    """
    Compute the value of the fstat for a range of parameters, with fixed
    amplitude over many realizations.

    """
    Tmaxyr = np.array([(p.toas.max() - p.toas.min())/3.16e7 for p in psr]).max()
    count = 0
    for ii in range(nreal):

        # draw parameter values
        gwtheta = np.arccos(np.random.uniform(-1, 1))
        gwphi = np.random.uniform(0, 2*np.pi)
        gwphase = np.random.uniform(0, 2*np.pi)
        gwinc = np.arccos(np.random.uniform(-1, 1))
        gwpsi = np.random.uniform(0, np.pi)

        
        gwmc = 10**np.random.uniform(7, 10)
        
        # determine distance in order to keep strain fixed
        gwdist = 2 * (gwmc*4.9e-6)**(5/3) * (np.pi*freq)**(2/3) / h

        # convert back to Mpc
        gwdist /= 1.0267e14
        
        # check for fixed sky location
        if theta is not None:
            gwtheta = theta
        if phi is not None:
            gwphi = phi
        if inc is not None:
            gwinc = inc
        if dist is not None:
            gwdist = dist
            gwmc = ((gwdist*1.0267e14)/2/(np.pi*freq)**(2/3)*h)**(3/5)/4.9e-6
            
            
        inducedRes = PALutils.createResidualsFast(model.psr, gwtheta, gwphi, gwmc, gwdist,
                            freq, gwphase, gwpsi, gwinc, evolve=False, phase_approx=False,
                            psrTerm=False)
        
        # create residuals 
        residuals = []
        for ct,p in enumerate(psr):
 
            # replace residuals in pulsar object
            noise = np.random.randn(len(p.residuals)) * np.sqrt(p.Nvec)
            residuals.append(np.dot(R[ct], noise+inducedRes[ct]))

    
        # compute f-statistic
        fpstat = model.fpStat(residuals, freq)
        
        # check to see if larger than in real data
        if detect:
            if ptSum(npsr, fpstat) < 1e-4:
                count += 1
        else:
            if fpstat > fstat_ref:
                count += 1

    # now get detection probability
    detProb = count/nreal
    
    #print freq, h, detProb

    return detProb - 0.95

In [21]:
# set up the pulsar model
model = PALmodels.PTAmodels(args.h5file, pulsars=[args.psr])

# if including ECORR, use incJitterEquad=True and separateJitterByFreq=True
fullmodel = model.makeModelDict(incRedNoise=True, noiseModel='powerlaw',
                                separateEfacsByFreq=True, separateEquadsByFreq=True,
                                incJitterEquad=True, separateJitterByFreq=True,
                                incEquad=True, nfreqs=50, ndmfreqs=50,  likfunc='mark6')

model.initModel(fullmodel, memsave=True, fromFile=False, verbose=True, write='no')

Requested to re-create the Auxiliaries
Creating Auxiliaries for J0613-0200


In [22]:
# read in noise model
chain = np.loadtxt('{}/chain_1.txt'.format(args.noisedir))
burn = int(0.25 * chain.shape[0])
# Get MAP parameters
maxpars = np.zeros(model.dimensions)
for ii in range(model.dimensions):
    maxpars[ii] = bu.getMax(chain[burn:,ii])

In [23]:
# begin Fp statistic calculations

fmin = fmin # These were defined earlier, just reminding you :)
fmax = fmax
nf = 300
fvec = np.logspace(np.log10(fmin), np.log10(fmax), nf)
fvec

array([8.00000000e-05, 8.06786425e-05, 8.13630420e-05, 8.20532473e-05,
       8.27493076e-05, 8.34512726e-05, 8.41591924e-05, 8.48731175e-05,
       8.55930988e-05, 8.63191878e-05, 8.70514362e-05, 8.77898963e-05,
       8.85346208e-05, 8.92856628e-05, 9.00430759e-05, 9.08069141e-05,
       9.15772321e-05, 9.23540846e-05, 9.31375273e-05, 9.39276159e-05,
       9.47244068e-05, 9.55279569e-05, 9.63383236e-05, 9.71555647e-05,
       9.79797384e-05, 9.88109036e-05, 9.96491197e-05, 1.00494446e-04,
       1.01346944e-04, 1.02206673e-04, 1.03073696e-04, 1.03948073e-04,
       1.04829868e-04, 1.05719143e-04, 1.06615962e-04, 1.07520388e-04,
       1.08432487e-04, 1.09352324e-04, 1.10279963e-04, 1.11215471e-04,
       1.12158916e-04, 1.13110363e-04, 1.14069882e-04, 1.15037540e-04,
       1.16013407e-04, 1.16997553e-04, 1.17990047e-04, 1.18990960e-04,
       1.20000364e-04, 1.21018331e-04, 1.22044933e-04, 1.23080245e-04,
       1.24124338e-04, 1.25177289e-04, 1.26239172e-04, 1.27310063e-04,
      

In [24]:
# initialize noise parameters in the model
model.setPsrNoise(maxpars)
model.updateTmatrix(maxpars)
model.mark6LogLikelihood(maxpars, incJitter=True)

-7426.859082390995

In [25]:
# get the Fp stats for the data
fp = np.zeros(nf)
for ii in range(nf):
    fp[ii] = model.fpStat([model.psr[0].residuals], fvec[ii])

np.savetxt(args.outdir + "{}_FpStat.txt".format(args.run_name), (fvec, fp))

In [26]:
# first create projection matrix and run likelihood to set noise covariance matrices
R = [PALutils.createRmatrix(p.Mmat, np.sqrt(p.Nvec)) for p in model.psr]
model.mark6LogLikelihood(maxpars, incJitter=True)

-7426.859082390995

In [27]:
## THE HEART OF THE CODE ##
# simulate sources using upperLimitFunc, 
# ramping up the strain amplitude until a 95% detection rate is passed

print "Starting upper limit analysis"

nf = args.frqres
print(nf)
nreal = args.nreal         #should be at least 1000
maxstrain = 5e-7     #chosen scientifically arbitrarily
fvec = np.logspace(np.log10(fmin), np.log10(fmax), nf)
h_up = np.zeros(nf)

Starting upper limit analysis
50


In [ ]:
for ii in range(nf):
    hlow = 1e-14
    hhigh = 1e-9
    xtol = 1e-14
    fp = model.fpStat([model.psr[0].residuals], fvec[ii])
    inRange = False
    while not inRange:
        try:
            h_up[ii] = brentq(upperLimitFunc, hlow, hhigh, xtol=xtol,
                              args=(model.psr, R, fp, fvec[ii], nreal))
            inRange = True
        except ValueError:
            if hhigh < maxstrain:
                hhigh *= 2
            else:
                h_up[ii] = hhigh
                inRange = True
    
    print('[{0} percent]Upper limit at {1} = {2}'.format((ii*100)/(nf), fvec[ii], h_up[ii]))

In [ ]:
run_name = ""
if hasattr(args, 'run_name'):
    run_name = args.run_name
else:
    run_name = 'PSR' + args.psr + '_' + str(args.frqres) + 'sample'

print "run name: {}".format(args.run_name)

# save the limit curve to this directory
np.savetxt(args.outdir + '{}.txt'.format(run_name), (fvec, h_up))
